# Question 1

In [74]:
import numpy as np
import scipy.linalg as la

In [99]:
def qr_decompose(A):
    
    m,n = A.shape
    Q = np.copy(A)
    R = np.zeros((n,n))
    
    for i in range(n):
        R[i,i] = la.norm(Q[:,i])
        Q[:,i] = Q[:,i] / R[i,i]
        for j in range(i+1,n):
            R[i,j] = Q[:,j].T @ Q[:,i]
            Q[:,j] = Q[:,j] - R[i,j]*Q[:,i]
    
    return Q,R

In [121]:
# Test
A = np.random.random((6,4))
Q, R = qr_decompose(A)

np.allclose(np.triu(R),R)
np.allclose(Q.T @ Q, np.identity(4))
np.allclose(Q @ R, A)

Q,R = la.qr(A, mode='economic')

# Question 2

In [122]:
def abs_det(A):
    Q,R = qr_decompose(A)
    det = np.absolute(np.diag(R).prod())
    return det
    

In [127]:
# test
A = np.random.random((6,6))
la.det(A) - abs_det(A)

-2.0816681711721685e-17

# Question 3

In [163]:
def solve_Linear(A, b):
    m,n = A.shape
    if len(b) != n or len(b) != m or m != n:
        raise ValueError('A must be nxn matrix and b an nx1 vector')
    if np.linalg.matrix_rank(A) != n:
        raise ValueError('A must be invertible')
    
    Q,R = qr_decompose(A)
    y = Q.T @ b
    x = la.inv(R) @ y
    return x

# Question 4

In [218]:
def householder(A):
    sign = lambda x: 1 if x>- 0 else -1
    
    m,n = A.shape
    R = np.copy(A)
    Q = np.eye(m)
    
    for k in range(n):
        u = np.copy(R[k:,k])
        u[0] = u[0] + sign(u[0]) * la.norm(u)
        u = u / la.norm(u)
        R[k:,k:] = R[k:,k:] - 2 * np.outer(u, (u.T @ R[k:,k:])) # apply reflection to R
        Q[k:,:] = Q[k:,:] - 2 * np.outer(u,(u.T @ Q[k:,:])) # apply reflection to Q
        
    return Q.T, R

In [220]:
# test
A = np.random.random((5,3))
Q,R = householder(A)
print(A.shape, Q.shape, R.shape)
np.allclose(Q @ R, A)

(5, 3) (5, 5) (5, 3)


True

# Question 5

In [247]:
def Hessenberg(A):
    sign = lambda x: 1 if x>- 0 else -1
    
    m,n = A.shape
    H = np.copy(A)
    Q = np.eye(m)
    for k in range(n-2):
        u = np.copy(H[k+1:,k])
        u[0] = u[0] + sign(u[0]) * la.norm(u)
        u = u / la.norm(u)
        H[k+1:,k:] = H[k+1:,k:] - 2 * np.outer(u, u.T @ H[k+1:,k:])
        H[:,k+1:] = H[:,k+1:] - 2 * np.outer(H[:,k+1:] @ u, u.T)
        Q[k+1:,:] = Q[k+1:,:] - 2 * np.outer(u, u.T @ Q[k+1:,:])
        
    return H,Q

In [248]:
A = np.random.random((8,8))
#H,Q = la.hessenberg(A, calc_q=True)

H,Q = Hessenberg(A)

In [249]:
np.allclose(np.triu(H,-1), H)

True

In [250]:
np.allclose(Q @ H @ Q.T, A)

False